In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader('Speech.txt')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=10)
docs=text_splitter.split_documents(documents)

In [4]:
embeddings=OllamaEmbeddings(model='gemma2:2b')
db=FAISS.from_documents(docs,embeddings)
db

In [7]:
## querying
query='what does the speaker believe is the main reason the unites states should enter the war? '
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [9]:
## Retriver

retriver=db.as_retriever()
docs=retriver.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [10]:
# Similarity Search with score

docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='0f0c61cb-e544-4cf5-8036-ab3f3b98fbd9', metadata={'source': 'Speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  33237.973),
 (Document(id='db7bbc58-4bb3-4acb-b2ef-5b108b9eb473', metadata={'source': 'Speech.txt'}, p

In [11]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.8266679644584656,
 -1.0602868795394897,
 -2.8650262355804443,
 1.615618109703064,
 -2.1306941509246826,
 -1.4909007549285889,
 4.437129497528076,
 -1.0005172491073608,
 3.998383045196533,
 -4.546699523925781,
 1.1301507949829102,
 -3.8543918132781982,
 -0.5859490633010864,
 -3.2697927951812744,
 -1.759046196937561,
 -0.5078995227813721,
 -2.3396522998809814,
 0.4724307358264923,
 0.44286781549453735,
 -3.987098455429077,
 -1.3939385414123535,
 0.868115246295929,
 -1.2974379062652588,
 3.202374219894409,
 -1.5651613473892212,
 -2.010812520980835,
 5.326577186584473,
 0.47241395711898804,
 -0.02131599932909012,
 -0.09557643532752991,
 -0.7498753070831299,
 -2.1552963256835938,
 1.2154316902160645,
 0.5388311147689819,
 -1.0352344512939453,
 2.744375705718994,
 -1.347603440284729,
 -3.165026903152466,
 0.7476322650909424,
 -0.4983016550540924,
 -1.4394211769104004,
 0.2551140785217285,
 1.2900903224945068,
 -1.45102059841156,
 -4.084743022918701,
 -2.2498040199279785,
 2.56558871269226

In [12]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='0f0c61cb-e544-4cf5-8036-ab3f3b98fbd9', metadata={'source': 'Speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='db7bbc58-4bb3-4acb-b2ef-5b108b9eb473', metadata={'source': 'Speech.txt'}, page_content='…\n

In [13]:
### Saving And Loading
db.save_local('faiss_index')

In [16]:
new_db=FAISS.load_local('faiss_index',embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)
docs

[Document(id='0f0c61cb-e544-4cf5-8036-ab3f3b98fbd9', metadata={'source': 'Speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='db7bbc58-4bb3-4acb-b2ef-5b108b9eb473', metadata={'source': 'Speech.txt'}, page_content='…\n